In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 15]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))

[ 1  2  3  4  5  6  7  8  9 10 11] [956483     15 481243  10597   4847 468586      4 223743    652   2579
     28]
(array([ 1,  3,  4,  5,  6,  8,  9, 10, 11]), array([956483, 481243,  10597,   4847, 468586, 223743,    652,   2579,
           28]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '3' encode 1
defense '4' encode 2
defense '5' encode 3
defense '6' encode 4
defense '8' encode 5
defense '9' encode 6
defense '10' encode 7
defense '11' encode 8


In [5]:
import numpy as np
from sklearn.cluster import KMeans

def stratified_cluster_split(X, y, test_size=0.5, random_state=42):
    X_sub_list = []
    y_sub_list = []
    X_rest_list = []
    y_rest_list = []

    classes = np.unique(y)

    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        X_c = X[y == c]
        y_c = y[y == c]

        kmeans = KMeans(n_clusters=2, random_state=random_state)
        cluster_labels = kmeans.fit_predict(X_c)

        cluster_0_idx = np.where(cluster_labels == 0)[0]
        cluster_1_idx = np.where(cluster_labels == 1)[0]

        if len(cluster_0_idx) <= len(cluster_1_idx):
            base_idx = cluster_0_idx
        else:
            base_idx = cluster_1_idx

        n_select = int(len(X_c) * test_size)
        if n_select == 0:
            continue

        if len(base_idx) >= n_select:
            selected_idx = rng.choice(base_idx, size=n_select, replace=False)
        else:
            extra_needed = n_select - len(base_idx)
            other_idx = np.setdiff1d(np.arange(len(X_c)), base_idx)
            extra_idx = rng.choice(other_idx, size=extra_needed, replace=False)
            selected_idx = np.concatenate([base_idx, extra_idx])

        rest_idx = np.setdiff1d(np.arange(len(X_c)), selected_idx)

        X_sub_list.append(X_c[selected_idx])
        y_sub_list.append(y_c[selected_idx])
        X_rest_list.append(X_c[rest_idx])
        y_rest_list.append(y_c[rest_idx])

    X_sub = np.concatenate(X_sub_list, axis=0)
    y_sub = np.concatenate(y_sub_list, axis=0)
    X_rest = np.concatenate(X_rest_list, axis=0)
    y_rest = np.concatenate(y_rest_list, axis=0)

    return X_rest, X_sub, y_rest, y_sub


_, X_1, _, y_1 = stratified_cluster_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, random_state=42)


In [6]:
X_1.shape

(21482, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt1 = DecisionTreeClassifier()
# dt1.fit(X_1, y_1)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 3 4 5] [   388 230310    155   7900]
(388, 42) (388,)
Save 1 to baseline_Def1.npy
(230310, 42) (230310,)
Save 3 to baseline_Def3.npy
(155, 42) (155,)
Save 4 to baseline_Def4.npy
(7900, 42) (7900,)
Save 5 to baseline_Def5.npy
Execution Time: 0.371997 seconds
(716259,)
[ 1  3  4  5  6 10] [218650 265088      1   1097 225200   6223]
(218650, 42) (218650,)
Save 1 to BIM_Def1.npy
(265088, 42) (265088,)
Save 3 to BIM_Def3.npy
(1, 42) (1,)
Save 4 to BIM_Def4.npy
(1097, 42) (1097,)
Save 5 to BIM_Def5.npy
(225200, 42) (225200,)
Save 6 to BIM_Def6.npy
(6223, 42) (6223,)
Save 10 to BIM_Def10.npy
Execution Time: 3.729291 seconds
(716259,)
[ 1  3  4  5  6 10] [ 22319 248197      1   2909 441258   1575]
(22319, 42) (22319,)
Save 1 to FGSM_Def1.npy
(248197, 42) (248197,)
Save 3 to FGSM_Def3.npy
(1, 42) (1,)
Save 4 to FGSM_Def4.npy
(2909, 42) (2909,)
Save 5 to FGSM_Def5.npy
(441258, 42) (441258,)
Save 6 to FGSM_Def6.npy
(1575, 42) (1575,)
Save 10 to FGSM_Def10.npy
Execution Time: 3.691089

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 4 5] [226752   5589   6412]
(226752, 42) (226752,)
Save 3 to baseline_Def3.npy
(5589, 42) (5589,)
Save 4 to baseline_Def4.npy
(6412, 42) (6412,)
Save 5 to baseline_Def5.npy
Execution Time: 0.968010 seconds
(716259,)
[ 1  3  5  6  8 10] [440176 269597   5734    693      1     58]
(440176, 42) (440176,)
Save 1 to BIM_Def1.npy
(269597, 42) (269597,)
Save 3 to BIM_Def3.npy
(5734, 42) (5734,)
Save 5 to BIM_Def5.npy
(693, 42) (693,)
Save 6 to BIM_Def6.npy
(1, 42) (1,)
Save 8 to BIM_Def8.npy
(58, 42) (58,)
Save 10 to BIM_Def10.npy
Execution Time: 2.422371 seconds
(716259,)
[ 1  3  5  6  8 10] [234430 234429  22803 224481      1    115]
(234430, 42) (234430,)
Save 1 to FGSM_Def1.npy
(234429, 42) (234429,)
Save 3 to FGSM_Def3.npy
(22803, 42) (22803,)
Save 5 to FGSM_Def5.npy
(224481, 42) (224481,)
Save 6 to FGSM_Def6.npy
(1, 42) (1,)
Save 8 to FGSM_Def8.npy
(115, 42) (115,)
Save 10 to FGSM_Def10.npy
Execution Time: 2.371861 seconds
(716259,)
[ 1  3  5  6  8 10] [440176 269597   5734

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt1.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Cluster/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Cluster/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Cluster/WUSTL_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_Cluster.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,388,0,230310,155,7900,0,0,0,0,0,0,238753
1,XGB1,218650,0,265088,1,1097,225200,0,0,0,6223,0,716259
2,XGB1,22319,0,248197,1,2909,441258,0,0,0,1575,0,716259
3,XGB1,218650,0,265088,1,1097,225200,0,0,0,6223,0,716259
4,XGB1,4495,0,18264,110,8758,682569,0,0,0,2063,0,716259
5,XGB1,48722,0,394911,33,3024,252649,0,0,0,16920,0,716259
6,XGB1,1164,0,690950,465,23680,0,0,0,0,0,0,716259
7,XGB1,72986,0,455267,6771,57508,0,0,123691,36,0,0,716259
8,XGB1,2184,0,269852,0,16,441846,0,0,0,2361,0,716259
9,XGB1,31956,0,246028,22,2424,432714,0,0,0,3115,0,716259
